<a href="https://colab.research.google.com/github/xjseabrum/comp_disc_S22_proj/blob/main/setup.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os as os

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
os.chdir("/content/gdrive/MyDrive/comp_proj")

In [ ]:
!pip install -U sentence-transformers
!pip install -U numpy
!pip install kneed

In [ ]:
# A Python interface to the Penn Discourse Treebank 2

!git clone https://github.com/cgpotts/pdtb2.git

In [ ]:
!ls

In [ ]:
# locate to the project dir
%cd /content/gdrive/MyDrive/comp_proj/pdtb2/

In [ ]:
# Unzip the csv file
!unzip pdtb2.csv.zip

In [ ]:
!python3 __init__.py

In [ ]:
#import statements
from sklearn.cluster import KMeans
import numpy as np
from pdtb2 import CorpusReader, Datum
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import time
from collections import Counter
import pandas as pd

# Data Extraction and Preprocessing

In [ ]:
# attribute reference http://compprag.christopherpotts.net/swda.html
iterator = CorpusReader('pdtb2.csv').iter_data(display_progress=False)
for _ in range(17): next(iterator)
d = next(iterator)
d.Relation

In [ ]:
from sentence_transformers import SentenceTransformer
sentences = ["This is an example sentence", "Each sentence is converted"]

model = SentenceTransformer('all-mpnet-base-v2')
embeddings = model.encode(sentences)
print(embeddings)

In [ ]:
#Read in Data
X_sentences = []
y = []
rel_tag = []
iterator = CorpusReader('pdtb2.csv').iter_data(display_progress=False)
try:
    while(True):
        d = next(iterator)
        arg1 = d.Arg2_RawText.lower()
        arg2 = d.Arg2_Attribution_RawText.lower()
        relation = d.Relation.lower()
        connection = None
        if relation == "Explicit":
            connection = d.ConnHead
            X_sentences.append([d.Arg1_RawText,d.ConnHead,d.Arg2_RawText])
            y.append(d.ConnHead)
            rel_tag.append(d.ConnHeadSemClass1)
except:
    print("end")

In [ ]:
# arg1 sentence and arg2 sentence
X_sentences[0]

In [ ]:
#connective phrase
y[0]

In [ ]:
len(X_sentences)

In [ ]:
# extract features using bert sentence transformers and concatenate two embedings.
# reference : https://huggingface.co/sentence-transformers/bert-base-nli-mean-tokens
# save features to file because it cost a lot of time extracting the features

n = len(X_sentences)
with open('features.npy', 'wb') as f:
  c = 0
  start = time.time()
  for sentences in X_sentences:
    feature = np.concatenate(model.encode(sentences))
    np.save(f, feature)
    if c % 1000 == 0:
      print(str(c) + "/" + str(n) + " complete.")
    if c % n == 0:
      print(str(c) + "/" + str(n) + " complete.")
    c += 1
  end = time.time()
  print("Total runtime: {} seconds".format(round(end - start, 2)))

In [ ]:
def view_labels(X, X_sentences, algo_labels, u):
  # algo_labels is labels as found by the algorithm in 1D array form
  # u is number of clusters

  arg1s = [x[0] for x in X_sentences]
  arg2s = [x[2] for x in X_sentences]
  arg_conn = [x[1] for x in X_sentences]
  arg_label = [y for y in rel_tag]
  df = pd.DataFrame([arg1s,arg2s,arg_conn,algo_labels, arg_label]).transpose()
  df.columns = ['arg1','arg2','connective', 'algo', 'pdtb_tag']

  df[['primary', 'secondary', 'tertiary']] = df.pdtb_tag.str.split('.', expand=True)

  labels = []
  for label in range(u):
    df2 = df[df['algo'] == label]
    tmp_list1 = set(df2["connective"])
    [x, y] = [label, [tmp_list1]]
    labels.append([x,y])

  connective_to_label = []
  for word in range(len(list(set(df["connective"])))):
    connective = list(set(df["connective"]))[word] 
    df2 = df[df["connective"] == connective]
    tmp_list2 = set(df2["algo"])
    [x,y] = [connective, [tmp_list2]]
    connective_to_label.append([x,y])

  return df, labels, connective_to_label